In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import (LinearRegression,
                                  RidgeCV,
                                  LassoCV,
                                  ElasticNetCV)
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from uoineuro.tuning_utils import create_strf_design

%matplotlib inline

In [2]:
data = h5py.File('/storage/data/dmr/dmr_strf_data.h5', 'r')

In [3]:
n_electrodes = len(data)
stim = data['Wave1/stim'][:]
n_samples, n_features = stim.shape
resp = np.zeros((n_electrodes, n_samples))

n_frames = 20
train_frac = 0.9

In [4]:
for electrode in range(n_electrodes):
    group_name = 'Wave' + str(electrode + 1)
    group = data[group_name]
    resp[electrode] = np.squeeze(group['resp'][:])

In [5]:
X, Y = create_strf_design(stim, resp, n_frames)

In [6]:
strfs = np.zeros((n_electrodes, n_frames, n_features))
train_scores = np.zeros(n_electrodes)
test_scores = np.zeros(n_electrodes)

In [7]:
for electrode in range(n_electrodes):
    print(electrode)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, Y[electrode], train_size=train_frac)

    centering = X_train.mean(axis=0, keepdims=True)
    X_train -= centering
    X_test -= centering

    ridge = RidgeCV(alphas=np.logspace(3, 6, num=300),
                    normalize=False,
                    fit_intercept=False,
                    cv=None).fit(X_train, y_train)
    print(ridge.alpha_)
    strfs[electrode] = ridge.coef_.reshape((20, 96))
    train_scores[electrode] = ridge.score(X_train, y_train)
    test_scores[electrode] = ridge.score(X_test, y_test)
    print('---')

0
629986.2978380694
---
1
659778.2479539546
---
2
615598.6501483514
---
3
615598.6501483514
---
4
601539.5880274833
---
5
629986.2978380694
---
6
644710.2107323869
---
7
601539.5880274833
---
8
574377.3748826587
---
9
601539.5880274833
---
10
587801.6072274912
---
11
615598.6501483514
---
12
615598.6501483514
---
13
615598.6501483514
---
14
644710.2107323869
---
15
629986.2978380694
---
16
587801.6072274912
---
17
601539.5880274833
---
18
561259.7255954966
---
19
601539.5880274833
---
20
850679.3416364171
---
21
601539.5880274833
---
22
812267.350047972
---
23
707128.4778942997
---
24
793716.7617217558
---
25
690979.0545715645
---
26
954845.5102782303
---
27
793716.7617217558
---
28
870561.2479099856
---
29
629986.2978380694
---
30
870561.2479099856
---
31
615598.6501483514
---
32
831251.4989890632
---
33
812267.350047972
---
34
870561.2479099856
---
35
775589.8323389002
---
36
793716.7617217558
---
37
812267.350047972
---
38
775589.8323389002
---
39
757876.886362593
---
40
757876.8863

In [20]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, Y[62], train_size=train_frac)

In [32]:
ridge = RidgeCV(alphas=np.linspace(1.05e6, 1.1e6, 300),
                normalize=False,
                fit_intercept=False,
                cv=None).fit(X_train, y_train)

In [36]:
strfs[62] = ridge.coef_.reshape((20, 96))
train_scores[62] = ridge.score(X_train, y_train)
test_scores[62] = ridge.score(X_test, y_test)

In [45]:
f = h5py.File('/home/psachdeva/fits/uoineuro/tuning/dmr/ecog_dmr.h5', 'w')
f['data/stim'] = stim
f['data/resp'] = resp
f['ridge/strfs'] = strfs
f['ridge/train_scores'] = train_scores
f['ridge/test_scores'] = test_scores
f.close()